### Variables:
status --> whether ticket is sold/unsold \
ownership --> the owner of the specific ticket, useful to verify ownership later \
\
ticket_id --> a unique ID for each generated ticket which is used to check for duplicate tickets 




### mineBlock() Function - use of Nonce in a block
The mineBlock() function implements a proof of work algorithm. When we are creating a block, we initialize a variable in the Block class called 'nonce' - number only used once. 

This nonce is a part of the class variables, hence it is included in the block string. So, the hash generated is dependent on the 'nonce'. In the algorithm, we have some preset conditions set by the blockchain, which act as a **consensus mechanism**, which is the hash of the block starting with '0000' in our case. 

### Importing Libraries:
hashlib: for generating cryptographic hash for blocks\
datetime: for timestamping blocks\
time: for calculating time difference between block creation\
secrets, string: for generating secret keys\
hmac: for implementing HMAC verification\
json: for creating strings in a specific pattern

In [4]:
import hashlib
import hmac
import json
import datetime as dt
import time as time
import secrets
import string

class Block:
    # class for creating blocks

    def __init__(self,index,timestamp,transactions,previous_hash,nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.transactions_data = ""
        for tr in self.transactions:
            i = tr["data"]
            self.transactions_data += i["ID"] + i["status"] + i["ownership"] 
        self.hash = self.HashCalculator()
    
    def HashCalculator(self): 
    # function to create hashes for an input of data

        transactions_info = str(self.index) + str(self.timestamp) + str(self.transactions_data) + self.previous_hash + str(self.nonce)
        string_data = json.dumps(transactions_info, sort_keys=True).encode()
        return hashlib.sha256(string_data).hexdigest() 
    
class BlockChain:
    # class for creating the blockchain

    def __init__(self):

        self.idx = 1 #indexing for added block
        self.chain = [] #starting chain with genesis block
        self.transactions_pool = [] #to store transactions yet to be included in the blockchain
        self.ledger = []
        self.last_block_time = time.time()
        self.create_genesis_block()

    def create_genesis_block(self):
        #creating genesis block

        genesis_block = Block(0,dt.datetime.now(),[{"data":{"ID": "0", "status": "N/A", "ownership": "N/A"},\
                                                    "type":"genesis_creation"}],"0",0) 
        self.chain.append(genesis_block)

    def create_transaction(self,data,type,hmac,buy_or_sell):
        #function to create a transaction

        return {"data": data, "type": type, "hmac": hmac, "buy/sell": buy_or_sell}

    def verifyTransaction(self,transaction,secret_key):
        #HMAC Verification for the transaction

        transaction_data = json.dumps(transaction["data"],sort_keys=True)
        recalculate_hmac = generate_HMAC(transaction_data,secret_key)

        if recalculate_hmac == transaction['hmac']:
            return True
        else:
            return False
        
    def check_create_block_criteria(self):
        #function to check if a block must be created

        if len(self.transactions_pool) >= 5 or (time.time() - self.last_block_time) >= 600:
            block = self.createBlock(dt.datetime.now(),self.transactions_pool)
            self.mineBlock(block)
            bool_accept = self.accept_or_reject_block(block)

            if bool_accept:
                self.transactions_pool = [] #resetting transaction pool
                self.last_block_time = time.time()
        
    def add_transaction(self,transaction,secret_key):
        #function to add a new transaction to unconfirmed transactions

        if self.verifyTransaction(transaction,secret_key):
            self.transactions_pool.append(transaction)
            self.ledger.append(transaction)
        
        if self.transactions_pool:
            self.check_create_block_criteria()
        
    def createBlock(self,timestamp,transactions):
        #function to create a block

        prev_hash = self.previous_block.hash 
        block = Block(self.idx,timestamp,transactions,prev_hash,nonce=0)
        return block  

    def mineBlock(self,block):
        #Proof of Work algorithm 

        block.nonce = 0
        hash = block.HashCalculator()
        while not hash.startswith('00'):
            block.nonce += 1
            hash = block.HashCalculator()
        return hash

    def validate_ProofOfWork(self,block):
        #Validation for the Proof of Work - A Consensus mechanism to check for eligibility 
        #to add the proposed block

        block.hash = self.mineBlock(block)
        return (block.hash.startswith('00') and block.previous_hash == self.chain[-1].hash)
    
    def accept_or_reject_block(self,block):
        #Accepting or rejecting blocks according to validation 

        if self.validate_ProofOfWork(block):
            self.chain.append(block)
            self.idx += 1 #updating index for next block  
            return True #to indicate block has been accepted
        
        return False
    
    @property
    def previous_block(self):
        return self.chain[-1]
    
def generate_HMAC(message,secret_key):
    return hmac.new(secret_key.encode(),message.encode(),hashlib.sha256).hexdigest()

#maintaining a user secret key database
user_key_database = {}

def generate_secret_key():
    #generating secret key for a user account

    length = 10
    options = string.ascii_letters + string.digits
    generated_key = ''.join(secrets.choice(options) for i in range(length))

    if generated_key not in user_key_database.values():
        return generated_key
    else:
        generate_secret_key()

user_key_database["organiser"] = generate_secret_key()
organiser_secret_key = user_key_database["organiser"]

blockchain = BlockChain()


''' CREATING TICKETS IN THE BLOCKCHAIN'''

def create_ticket(date,time,seat_no,secret_key):
    #function to create a ticket

    ticket_ID = date + time + seat_no 
    status = "unsold"
    ownership = "organiser"
    ticket_info = {
        "date": date,
        "time": time,
        "seat_number": seat_no,
        "status": status,
        "ownership": ownership,
        "ID": ticket_ID
    }

    data_for_hmac = json.dumps(ticket_info,sort_keys=True)
    code_hmac = generate_HMAC(data_for_hmac,secret_key)
    transaction_create_ticket = blockchain.create_transaction(ticket_info,"create_ticket",code_hmac,"None")

    blockchain.add_transaction(transaction_create_ticket,secret_key)


no_of_tickets_per_day = int(input("Enter number of tickets per day to be sold: "))
no_of_days = int(input("Enter number of days of event"))
total_no_of_tickets = no_of_tickets_per_day * no_of_days
# event_directory = {} 
for day_no in range(1, no_of_days+1):
    date = input(f"enter date for day {day_no} (Enter in DD-MM-YYYY format)")
    time_ = input(f"enter time for day {day_no} (Enter in 24-hr HH:MM format)")

    for i in range(1, no_of_tickets_per_day+1):
        seat_number = str(i)
        create_ticket(date,time_,seat_number,organiser_secret_key)


print("Number of Blocks created",blockchain.idx)
for i in blockchain.chain:
    print(i.__dict__)
        

Number of Blocks created 13
{'index': 0, 'timestamp': datetime.datetime(2024, 4, 21, 4, 20, 0, 500921), 'transactions': [{'data': {'ID': '0', 'status': 'N/A', 'ownership': 'N/A'}, 'type': 'genesis_creation'}], 'previous_hash': '0', 'nonce': 0, 'transactions_data': '0N/AN/A', 'hash': '4e30b1a2723285f3dd5eeb735557a212c3ecc7d5a1df182a56cb6a8e73845e63'}
{'index': 1, 'timestamp': datetime.datetime(2024, 4, 21, 4, 20, 21, 553456), 'transactions': [{'data': {'date': '12-05-2024', 'time': '6:00', 'seat_number': '1', 'status': 'unsold', 'ownership': 'organiser', 'ID': '12-05-20246:001'}, 'type': 'create_ticket', 'hmac': '6f030895200d5d206e1a130f8edd7d6cab1e19ef1ba0239232358b3dbead8fef', 'buy/sell': 'None'}, {'data': {'date': '12-05-2024', 'time': '6:00', 'seat_number': '2', 'status': 'unsold', 'ownership': 'organiser', 'ID': '12-05-20246:002'}, 'type': 'create_ticket', 'hmac': '7a294a7b10d307ede516f5d121be82b8f611f483ebc620c213156617dbae6f83', 'buy/sell': 'None'}, {'data': {'date': '12-05-2024'

In [27]:
'''CREATING USER ACCOUNTS'''

def get_userID(name,gender,age):
    #function to generate the user ID for the user
    
    user_ID = name + gender + age

    if user_ID not in user_key_database:
        return user_ID
    print("User Exists")
    raise KeyError

def create_user_account():
    #function to create a user account with his details

    name = input("Enter name")
    gender = input("Enter gender")
    age = input("Enter age")
    UserID = get_userID(name, gender, age)
    secretKey = generate_secret_key()
    user_key_database[UserID] = secretKey

    return UserID, secretKey

def ticket_status_check(ticket_id,blkchn,status,ownership):
    #function to check if the queried ticket is available for sale 

    for i in reversed(blkchn.ledger):
        k = i["data"]
        if ticket_id == k["ID"]:
            #check status
            if k["status"] == status and k["ownership"] == ownership:
                return True,k
            return False,0
            

'''PURCHASE TICKETS'''

def purchase_ticket(user_id):
    #function to implement the ticket purchase from organiser process

    date = input("Which date is the event? (Enter in DD-MM-YYYY format)")
    time = input("What time is the event? (Enter in 24-hr HH:MM format)")
    seat_no = input("What seat number do you want?")
    
    ticket_id = date + time + seat_no
    available, ticket_inf = ticket_status_check(ticket_id,blockchain,"unsold","organiser")

    if available:
        ticket_inf["status"] = "sold"
        ticket_inf["ownership"] = user_id
        secretkey = user_key_database[user_id]
        data_for_hmac_p = json.dumps(ticket_inf,sort_keys=True)
        code_hmac_p = generate_HMAC(data_for_hmac_p,secretkey)
        transaction_purchase = blockchain.create_transaction(ticket_inf,"purchase_ticket",code_hmac_p,user_id)
        blockchain.add_transaction(transaction_purchase,secretkey)


'''SELL TICKETS'''

def sell_ticket(user_id,ticket_id):
    #function to implement the ticket sale to organiser process
    
    valid_owner, ticket_inf = ticket_status_check(ticket_id,blockchain,"sold",user_id)

    if valid_owner:
        ticket_inf["status"] = "unsold"
        ticket_inf["ownership"] = "organiser"
        secretkey = user_key_database[user_id]
        data_for_hmac_s = json.dumps(ticket_inf,sort_keys=True)
        code_hmac_s = generate_HMAC(data_for_hmac_s,secretkey)
        transaction_sell = blockchain.create_transaction(ticket_inf,"sell_ticket",code_hmac_s,user_id)
        blockchain.add_transaction(transaction_sell,secretkey)


'''VERIFY USER'S TICKET AT ENTRY'''

def verify_user_ticket(user_id,ticket_id):
    #function to verify user's ticket at entry

    valid_owner, ticket_inf = ticket_status_check(ticket_id,blockchain,"sold",user_id)

    return valid_owner

In [12]:
# Person A - Create and Purchase
id, key = create_user_account()
purchase_ticket(id)
print(blockchain.ledger[-1])


{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'AFemale27', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'e6cbe564396c56b3c00d593a0336309550deeab429950c48672e9dc252b4c3a7', 'buy/sell': 'AFemale27'}


#### Transaction for the purchase:
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'AFemale27', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'e6cbe564396c56b3c00d593a0336309550deeab429950c48672e9dc252b4c3a7', 'buy/sell': 'AFemale27'}

In [28]:
# Person A - Sell
id = "AFemale27"
ticket = "15-06-202419:0019"
sell_ticket(id,ticket)

print(blockchain.ledger[-1])

{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'unsold', 'ownership': 'organiser', 'ID': '15-06-202419:0019'}, 'type': 'sell_ticket', 'hmac': 'c7070112f662b9c5332e90fb00bdc824ba87b9a6c12ac26edeb8a7966cbca6be', 'buy/sell': 'AFemale27'}


#### Transaction for the sale:
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'unsold', 'ownership': 'organiser', 'ID': '15-06-202419:0019'}, 'type': 'sell_ticket', 'hmac': 'c7070112f662b9c5332e90fb00bdc824ba87b9a6c12ac26edeb8a7966cbca6be', 'buy/sell': 'AFemale27'}

In [29]:
# Person B - Create and Purchase the resold ticket
id, key = create_user_account()
purchase_ticket(id)
print(blockchain.ledger[-1])

{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'a0d17d0039fad581162d823c7228b5533ad66cb14fa209b433a0607ead87d9dd', 'buy/sell': 'BMale56'}


#### Transaction for the repurchase of the same ticket:
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'a0d17d0039fad581162d823c7228b5533ad66cb14fa209b433a0607ead87d9dd', 'buy/sell': 'BMale56'}

In [30]:
'''Viewing list of transactions for a user'''

def viewUser(blkchn,userid):
    for i in blkchn.ledger:
        if i["buy/sell"] == userid:
            print(i)

In [31]:
# Printing person A transactions
viewUser(blockchain,"AFemale27")

# Printing person B transactions
viewUser(blockchain,"BMale56")

{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'e6cbe564396c56b3c00d593a0336309550deeab429950c48672e9dc252b4c3a7', 'buy/sell': 'AFemale27'}
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'sell_ticket', 'hmac': 'c7070112f662b9c5332e90fb00bdc824ba87b9a6c12ac26edeb8a7966cbca6be', 'buy/sell': 'AFemale27'}
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'a0d17d0039fad581162d823c7228b5533ad66cb14fa209b433a0607ead87d9dd', 'buy/sell': 'BMale56'}


#### Output:
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'e6cbe564396c56b3c00d593a0336309550deeab429950c48672e9dc252b4c3a7', 'buy/sell': 'AFemale27'}\
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'sell_ticket', 'hmac': 'c7070112f662b9c5332e90fb00bdc824ba87b9a6c12ac26edeb8a7966cbca6be', 'buy/sell': 'AFemale27'}\
{'data': {'date': '15-06-2024', 'time': '19:00', 'seat_number': '19', 'status': 'sold', 'ownership': 'BMale56', 'ID': '15-06-202419:0019'}, 'type': 'purchase_ticket', 'hmac': 'a0d17d0039fad581162d823c7228b5533ad66cb14fa209b433a0607ead87d9dd', 'buy/sell': 'BMale56'}
##### First two are of user AFemale27
##### Third is of user BMale56


Error : once ownership changed, it is being changed in all transanction data.